In [1]:
import polars as pl
import datetime as dt

import os
import sys

new_path = "/".join(os.getcwd().split("/")[:-1])
sys.path.append(new_path)

print(sys.path)

rooms_csv = pl.read_csv(f"{new_path}/data/updated_core_rooms.csv", try_parse_dates=True)

['/home/shekhar/.local/share/uv/python/cpython-3.13.0-linux-x86_64-gnu/lib/python313.zip', '/home/shekhar/.local/share/uv/python/cpython-3.13.0-linux-x86_64-gnu/lib/python3.13', '/home/shekhar/.local/share/uv/python/cpython-3.13.0-linux-x86_64-gnu/lib/python3.13/lib-dynload', '', '/home/shekhar/shekhar/CUPIDApi/.venv/lib/python3.13/site-packages', '/home/shekhar/shekhar/CUPIDApi']


In [2]:
pl.Config.set_fmt_str_lengths(100000)  # for string columns, increases string length shown
pl.Config.set_tbl_cols(-1)           # show all columns
pl.Config.set_tbl_rows(100)  

polars.config.Config

In [ ]:
rooms_csv

In [4]:
duplicates_exist_rooms_csv = rooms_csv.select(
    pl.all().map_batches(lambda s: s.is_duplicated().sum()).name.suffix("_duplicate_count")
)

print(duplicates_exist_rooms_csv, len(rooms_csv))

shape: (1, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ core_room_id_d ┆ core_hotel_id_ ┆ lp_id_duplicat ┆ supplier_room ┆ supplier_name ┆ supplier_room │
│ uplicate_count ┆ duplicate_coun ┆ e_count        ┆ _id_duplicate ┆ _duplicate_co ┆ _name_duplica │
│ ---            ┆ t              ┆ ---            ┆ _count        ┆ unt           ┆ te_count      │
│ i64            ┆ ---            ┆ i64            ┆ ---           ┆ ---           ┆ ---           │
│                ┆ i64            ┆                ┆ i64           ┆ i64           ┆ i64           │
╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ 0              ┆ 2538704        ┆ 2538704        ┆ 87008         ┆ 2869056       ┆ 2271249       │
└────────────────┴────────────────┴────────────────┴───────────────┴───────────────┴───────────────┘ 2869056


In [29]:
map_ = {}
for row in rooms_csv.iter_rows(named=True):
    if row["core_hotel_id"] in map_:
        if row["lp_id"] != map_[row["core_hotel_id"]]:
            print(row)
            break
    else:
        map_[row["core_hotel_id"]] = row["lp_id"]
    # break
len(map_)

813834

In [27]:
reference_csv = pl.read_csv(f"{new_path}/data/referance_rooms-1737378184366.csv", try_parse_dates=True)

In [28]:
reference_csv.head()

hotel_id,lp_id,room_id,room_name
i64,str,i64,str
13484077,"""lp23e8ef""",1142730702,"""Double or Twin Room"""
13487663,"""lp6554de34""",1141927122,"""House"""
13462809,"""lp6556c3dc""",1142722063,"""Room"""
13530116,"""lp6555450b""",1141968275,"""Triple Room"""
13530071,"""lp6557a92c""",1142513784,"""Apartment"""


In [31]:
map_ = {}
for row in reference_csv.iter_rows(named=True):
    if row["hotel_id"] in map_:
        if row["lp_id"] != map_[row["hotel_id"]]:
            print(row)
            break
    else:
        map_[row["hotel_id"]] = row["lp_id"]
    # break
len(map_)

40011

In [6]:
duplicates_exist_reference_csv = reference_csv.select(
    pl.all().map_batches(lambda s: s.is_duplicated().sum()).name.suffix("_duplicate_count")
)

print(duplicates_exist_reference_csv)

shape: (1, 4)
┌────────────────────────┬───────────────────────┬────────────────────────┬────────────────────────┐
│ hotel_id_duplicate_cou ┆ lp_id_duplicate_count ┆ room_id_duplicate_coun ┆ room_name_duplicate_co │
│ nt                     ┆ ---                   ┆ t                      ┆ unt                    │
│ ---                    ┆ i64                   ┆ ---                    ┆ ---                    │
│ i64                    ┆                       ┆ i64                    ┆ i64                    │
╞════════════════════════╪═══════════════════════╪════════════════════════╪════════════════════════╡
│ 76608                  ┆ 76608                 ┆ 0                      ┆ 75856                  │
└────────────────────────┴───────────────────────┴────────────────────────┴────────────────────────┘


In [7]:
len(reference_csv), len(rooms_csv)

(100000, 2869056)

In [ ]:
merged_df = reference_csv.join(rooms_csv, on="lp_id", how="inner")
print(merged_df.head(2))

In [28]:
len(merged_df)

434165

In [ ]:
merged_df2 = reference_csv.join(rooms_csv, left_on="hotel_id", right_on="core_hotel_id", how="inner")
merged_df2


In [9]:
len(merged_df2)

19527

In [ ]:
merged_df3 = reference_csv.join(rooms_csv, left_on="room_id", right_on="core_room_id", how="inner")
merged_df3.head()

In [ ]:
len(merged_df3)


In [ ]:
rooms_csv["supplier_name"].unique()

polars.config.Config

In [10]:
for hotel in merged_df2["hotel_id"].unique():
    df_filtered = merged_df2.filter(pl.col("hotel_id") == hotel)
    print(f"DataFrame for hotel_id: {hotel}")
    # print(df_filtered)
    break
df_filtered

DataFrame for hotel_id: 112749


hotel_id,lp_id,room_id,room_name,core_room_id,lp_id_right,supplier_room_id,supplier_name,supplier_room_name
i64,str,i64,str,i64,str,i64,str,str
112749,"""lp655c163c""",59067,"""Double Room""",2264106,"""lp1b86d""",14585,"""Expedia""","""Room, 1 King Bed, Non Smoking"""
112749,"""lp655c163c""",59085,"""Executive Room""",2264106,"""lp1b86d""",14585,"""Expedia""","""Room, 1 King Bed, Non Smoking"""
112749,"""lp655c163c""",11274905,"""Small Double Room""",2264106,"""lp1b86d""",14585,"""Expedia""","""Room, 1 King Bed, Non Smoking"""
112749,"""lp655c163c""",11274906,"""Double or Twin Room""",2264106,"""lp1b86d""",14585,"""Expedia""","""Room, 1 King Bed, Non Smoking"""
112749,"""lp655c163c""",59067,"""Double Room""",2273572,"""lp1b86d""",14586,"""Expedia""","""Room, 2 Queen Beds, Non Smoking"""
112749,"""lp655c163c""",59085,"""Executive Room""",2273572,"""lp1b86d""",14586,"""Expedia""","""Room, 2 Queen Beds, Non Smoking"""
112749,"""lp655c163c""",11274905,"""Small Double Room""",2273572,"""lp1b86d""",14586,"""Expedia""","""Room, 2 Queen Beds, Non Smoking"""
112749,"""lp655c163c""",11274906,"""Double or Twin Room""",2273572,"""lp1b86d""",14586,"""Expedia""","""Room, 2 Queen Beds, Non Smoking"""
112749,"""lp655c163c""",59067,"""Double Room""",2284525,"""lp1b86d""",201100289,"""Expedia""","""Family Suite, 1 King Bed, Non Smoking"""


In [22]:
rooms_csv.filter(pl.col("core_hotel_id") == 235111)

core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name
i64,i64,str,i64,str,str
667943,235111,"""lp39667""",200251101,"""Expedia""","""Standard Room, 2 Queen Beds, Non Smoking"""
674967,235111,"""lp39667""",200251098,"""Expedia""","""Standard Room, 1 King Bed, Non Smoking"""
679667,235111,"""lp39667""",201429392,"""Expedia""","""Room, Accessible"""
683004,235111,"""lp39667""",200251103,"""Expedia""","""Standard Room, 1 King Bed, Accessible, Non Smoking"""
685563,235111,"""lp39667""",200251106,"""Expedia""","""Suite, 1 King Bed"""


In [23]:
reference_csv.filter(pl.col("hotel_id") == 235111)

hotel_id,lp_id,room_id,room_name
i64,str,i64,str
235111,"""lp656af16b""",23511112,"""King Room - Non-Smoking"""
235111,"""lp656af16b""",23511113,"""Queen Room with Two Queen Beds and Sofa Bed"""
235111,"""lp656af16b""",23511116,"""Queen Room with Two Queen Beds and Communication Assitance"""
235111,"""lp656af16b""",23511119,"""King Room"""
235111,"""lp656af16b""",23511120,"""King Room with Mobility Accessible Tub"""
235111,"""lp656af16b""",23511121,"""King Room with Roll-In Shower - Disability Access"""
235111,"""lp656af16b""",23511122,"""Queen Room with Bath Tub - Disability Access"""
235111,"""lp656af16b""",23511123,"""King Room with Sofa Bed"""
235111,"""lp656af16b""",23511124,"""Queen Room with Two Queen Beds"""


In [21]:
merged_df2["hotel_id"].unique()

hotel_id
i64
112749
116927
171130
172450
172833
175798
176914
177302
177894
